# Computer Vision

## Assignment - UNet

### Install

In [ ]:
%pip install segmentation_models --quiet
%pip install comet_ml --quiet

### Comet Config

In [ ]:
from comet_ml import Experiment

experiment = Experiment(project_name = "cv-deep")

### Imports

In [ ]:
from glob import glob
import time
import os
from os import path

from natsort import natsorted
from tqdm.auto import tqdm

import imageio
import numpy as np
from skimage import img_as_float32

import tensorflow as tf
import keras
from keras import callbacks
from keras.models import load_model
from segmentation_models import Unet


print(f"numpy=={np.__version__}")
print(f"tensorflow=={tf.__version__}")
print(f"keras=={keras.__version__}")

### Common

In [ ]:
def load_images(input_dir, height, width):
    image_dir = path.join(input_dir, f'images-{width}x{height}')
    image_paths = natsorted(glob(f'{image_dir}/*.png'))
    raw_images = np.array([
        img_as_float32(imageio.imread(path))
        for path in tqdm(image_paths, 'Reading in images')
    ])

    mask_dir = path.join(input_dir, f'masks-{width}x{height}')
    mask_paths = natsorted(glob(f'{mask_dir}/*.png'))
    raw_masks = np.array([
        img_as_float32(imageio.imread(path))
        for path in tqdm(mask_paths, 'Reading in masks')
    ])

    return raw_images, raw_masks

In [ ]:
def split_data(raw_images, raw_masks):
    N = len(raw_images)
    indices = np.random.permutation(N)

    train_end = int(0.7*N) + 1
    train_indices = indices[:train_end]

    validation_end = train_end + int(0.15*N)
    validation_indices = indices[train_end:validation_end]

    test_indices = indices[validation_end:]

    return (raw_images[train_indices], raw_masks[train_indices]),\
        (raw_images[validation_indices], raw_masks[validation_indices]),\
        (raw_images[test_indices], raw_masks[test_indices])

In [ ]:
def preprocess_images(raw_images, raw_masks):
    images = raw_images
    masks = raw_masks
    
    if len(masks.shape) == 3:
        masks = np.expand_dims(masks, axis=3)
        
    return images, masks

### Constants

In [ ]:
BACKBONE = 'vgg16'

assets_dir = './assets'
data_dir = './data'

default_input_size = [768, 1024]
default_input_dir = path.join(
    assets_dir,
    f'puzzle_corners_{default_input_size[1]}x{default_input_size[0]}')

time_now = int(time.time())

default_cp_path = path.join(data_dir, str(time_now), 'cp.ckpt')
default_model_path = path.join(data_dir, str(time_now), 'model.h5')

### Params

In [ ]:
params = {
    'augmented': True,
    'batch_size': 2,
    'epochs': 2,
    'input_size': default_input_size,
}

experiment.log_parameters(params)

### Setup

In [ ]:
def setup_args():
    return {
        'cp_path': default_cp_path,
        'input_dir': default_input_dir,
        'model_path': default_model_path,
        'update_model': None,
        'cpu': True
    }

### Train

In [ ]:
args = setup_args()

use_cpu = args['cpu']
if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

augmented = params['augmented']
batch_size = params['batch_size']
epochs = params['epochs']
input_size = params['input_size']

cp_path = args['cp_path']
input_dir = args['input_dir']
model_path = args['model_path']
update_model = args['update_model']

height, width = input_size

raw_images, raw_masks = load_images(input_dir, height, width)

images, masks = preprocess_images(raw_images, raw_masks)


In [ ]:
(train_x, train_y), (val_x, val_y), (test_x, test_y) =\
    split_data(raw_images, masks)
    
experiment.log_dataset_hash(train_x)

checkpoint_cb = callbacks.ModelCheckpoint(filepath=cp_path,
                                          save_weights_only=True,
                                          verbose=1)

if update_model and model_path:
    model = load_model(model_path)
else:
    model = Unet(BACKBONE,
                 encoder_weights='imagenet',
                 input_shape=(*input_size, 3))
    
assert model is not None, "Could not load model"

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(val_x, val_y),
          callbacks=[checkpoint_cb])

model.save(model_path)

### Comet Commit

In [ ]:
experiment.end()